# import necessary librairy
pandas to work with data as tables
numpy for number matrices
matplotlib to plot result
tensor flow for our deap learning


In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Loading our data and removing indesired column

In [3]:
dataframe = pd.read_csv("data.csv")
dataframe = dataframe.drop(["index", "price","sq_price"], axis=1) #removing indesired column
dataframe = dataframe[0:10] #we choose the first 10 rows to train our model
dataframe

,area,bathrooms
0,2104.0,3.0
1,1600.0,3.0
2,2400.0,3.0
3,1416.0,2.0
4,3000.0,4.0
5,1985.0,4.0
6,1534.0,3.0
7,1427.0,3.0
8,1380.0,3.0
9,1494.0,3.0


# Lets classify our data
We will classify ou data to two categories (good deal, and bad deal), by adding 1 in front of the good deal and 0 for the bad one

In [4]:
dataframe.loc[:, ("y1")] = [1, 1, 1, 0, 0, 1, 0, 1, 1, 1]
dataframe.loc[:, ("y2")] = dataframe["y1"] == 0 # y2 is the negation of y1
dataframe.loc[:, ("y2")] = dataframe["y2"].astype(int) #convert from Ture/False to 1/0
# y2 is redundant but learning to do it this way opens the door to multiclass classification
dataframe

,area,bathrooms,y1,y2
0,2104.0,3.0,1,0
1,1600.0,3.0,1,0
2,2400.0,3.0,1,0
3,1416.0,2.0,0,1
4,3000.0,4.0,0,1
5,1985.0,4.0,1,0
6,1534.0,3.0,0,1
7,1427.0,3.0,1,0
8,1380.0,3.0,1,0
9,1494.0,3.0,1,0


In [ ]:
Now that we have all our data in the dataframe, 
we will need to shap it in matrices to feed it to TensorFlow

In [5]:
inputX = dataframe.loc[:, ['area', 'bathrooms']].as_matrix()
inputY = dataframe.loc[:, ["y1", "y2"]].as_matrix()

In [6]:
inputX

array([[  2.10400000e+03,   3.00000000e+00],
       [  1.60000000e+03,   3.00000000e+00],
       [  2.40000000e+03,   3.00000000e+00],
       [  1.41600000e+03,   2.00000000e+00],
       [  3.00000000e+03,   4.00000000e+00],
       [  1.98500000e+03,   4.00000000e+00],
       [  1.53400000e+03,   3.00000000e+00],
       [  1.42700000e+03,   3.00000000e+00],
       [  1.38000000e+03,   3.00000000e+00],
       [  1.49400000e+03,   3.00000000e+00]])

In [7]:
inputY

array([[1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0]])

# Preparing the training parameters

In [10]:
# Parameters
learning_rate = 0.000001
training_epochs = 2000
display_step = 50
n_samples = inputY.size

And now to define the TensorFlow operations. Notice that this is a declaration step where we tell TensorFlow how the prediction is calculated. if we execute it, no calculation would be made. It just acknowledge that it now knows how to do the operation.

In [8]:
# TensorFlow, we will feed you an array of examples. Each example will 
# be an array of two float values (area, and number of bathrooms)
# "None" means we can feed you any number of examples
x = tf.placeholder(tf.float32, [None, 2])
# Notice we haven't fed it the values yes

w = tf.Variable(tf.zeros([2,2])) # Maintain a 2 x 2 float matrix for the weights
# that we'll keep updating through the training process (make the all zero
# to begin with)

b = tf.Variable(tf.zeros([2])) # also maintain two bias values
y_values = tf.add(tf.matmul(x,w), b) # the first step in calculating the prediction would be multiply
# the inputs matrix by the weights matrix then add the biases
y = tf.nn.softmax(y_values) # then we use softmax as an "activation function"
# that translates the numbers outputted by the previous layer into probability form
y_ = tf.placeholder(tf.float32, [None, 2]) # For training purposes, we will
# also feed you a matrix of labels

Let's specify our coast function and use Gradient Descent

In [11]:
# Coast function: Mean squared error
cost = tf.reduce_sum(tf.pow(y_ - y, 2)) / (2*n_samples)
# Gradient descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [13]:
# Initialize variables and TensorFlow session
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

And now for the actual training

In [16]:
for i in range(training_epochs):
    sess.run(optimizer, feed_dict={x: inputX, y_: inputY}) # Take a gradient 
    # descent step using our inputs and labels
    
    # That's all! The rest of the cell just outputs debug messages.
    # Display logs per epoch step
    
    if (i) % display_step == 0:
        cc = sess.run(cost, feed_dict={x: inputX, y_:inputY})
        print("Training step:", '%04d' % (i), "cost=", "%.9f" % cc)
print("Optimization Finished!")
training_cost = sess.run(cost, feed_dict={x: inputX, y_: inputY})
print("Training cost=", training_cost, "w=", sess.run(w), "b=",sess.run(b), '\n')

Training step: 0000 cost= 0.109534547
Training step: 0050 cost= 0.109534480
Training step: 0100 cost= 0.109534420
Training step: 0150 cost= 0.109534346
Training step: 0200 cost= 0.109534286
Training step: 0250 cost= 0.109534219
Training step: 0300 cost= 0.109534144
Training step: 0350 cost= 0.109534070
Training step: 0400 cost= 0.109534003
Training step: 0450 cost= 0.109533928
Training step: 0500 cost= 0.109533861
Training step: 0550 cost= 0.109533802
Training step: 0600 cost= 0.109533742
Training step: 0650 cost= 0.109533668
Training step: 0700 cost= 0.109533593
Training step: 0750 cost= 0.109533526
Training step: 0800 cost= 0.109533466
Training step: 0850 cost= 0.109533392
Training step: 0900 cost= 0.109533310
Training step: 0950 cost= 0.109533265
Training step: 1000 cost= 0.109533191
Training step: 1050 cost= 0.109533116
Training step: 1100 cost= 0.109533049
Training step: 1150 cost= 0.109532975
Training step: 1200 cost= 0.109532908
Training step: 1250 cost= 0.109532848
Training ste

Now the training is done, TensorFlow is now holding on to our trained model (Which is basically just the defined operations, plus the variables w and b that resulted from the training process).

# Challenge the model
Is a coast of 0.109532 good or bad? I have no idea. Let's use the model on our dataset to see how it does:

In [17]:
sess.run(y, feed_dict={x: inputX})

array([[ 0.71120936,  0.28879055],
       [ 0.66498959,  0.33501041],
       [ 0.73651242,  0.26348755],
       [ 0.6471585 ,  0.3528415 ],
       [ 0.78329271,  0.2167073 ],
       [ 0.70070404,  0.29929596],
       [ 0.65866917,  0.34133083],
       [ 0.64830232,  0.35169768],
       [ 0.64370334,  0.35629663],
       [ 0.65481079,  0.34518918]], dtype=float32)

if we trust it we can run it on other data not used to training (from row 11 to the last one)